In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

## 0. Задания модулей

**Задание 2.1**

Сколько записей событий находится в датасете? Ответ введите без пробелов и знаков препинания.

In [3]:
events = pd.read_csv('data\events.csv.zip')
events.sample(5)

,timestamp,visitorid,event,itemid,transactionid
988096,1440173905041,1032492,view,78268,NaN
445945,1434959217322,557809,view,335076,NaN
2360074,1436903709519,952246,addtocart,280058,NaN
1130412,1440947530384,994820,view,301947,NaN
10552,1433190434796,417972,view,445749,NaN


In [4]:
print('В датасете', len(events.index), 'записей')

В датасете 2756101 записей


**Задание 2.2**

Какие типы событий содержатся в датасете? Выберите все верные варианты.

In [5]:
events.event.unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

Типы событий:  

view (просмотр карточки товара)  
addtocart (добавление в корзину)  
transaction (покупка)

**Задание 2.3**

Сколько уникальных признаков товара есть в датасете? Ответ вводите без пробелов и знаков препинания.

In [6]:
item_properties_1 = pd.read_csv('data\item_properties_part1.csv')
item_properties_2 = pd.read_csv('data\item_properties_part2.csv')
item_properties =pd.concat([item_properties_1, item_properties_2], axis=0)
item_properties.sample(5)

,timestamp,itemid,property,value
4112593,1435460400000,272994,486,1154859 86628
8964761,1435460400000,18223,678,223745
8308599,1433041200000,192377,0,1008741 561431 508431
2336433,1435460400000,88992,810,1241026 1160361 n240.000 424566
1043436,1440903600000,139080,790,n37080.000


In [7]:
print('В датасете', item_properties.property.nunique(), 'признаков товара')

В датасете 1104 признаков товара


events — датасет с событиями. Колонки:

timestamp — время события  
visitorid — идентификатор пользователя  
event — тип события  
itemid — идентификатор объекта  
transactionid — идентификатор транзакции, если она проходила  
category_tree — файл с деревом категорий (можно восстановить дерево).  

category_id — идентификатор категорий  
parent_id — идентификатор родительской категории  
item_properties — файл с свойствами товаров.  

timestamp — момент записи значения свойства  
item_id — идентификатор объекта  
property — свойство, кажется, они все, кроме категории, захешированы  
value — значение свойства  

**Задание 4.1**

Какой процент продаж обеспечивают топовые товары (точка отсечения — 1 июля)? Для расчёта используйте топ-3 товаров по числу транзакций.

Ответ введите без знака процента.

In [8]:
events['datetime']=pd.to_datetime(events.timestamp, unit='ms')
events.datetime.max()

Timestamp('2015-09-18 02:59:47.788000')

In [9]:
top_products = events[(events.datetime<='2015-07-01') & (events.event=='transaction')].itemid.value_counts(normalize=True).iloc[:3].index

In [10]:
persent_3 = events[(events.datetime>='2015-07-01') & (events.event=='transaction')].itemid.value_counts(normalize=True).loc[[top_products[0],top_products[2]]].sum()*100
print('Процент продаж топ трех товаров', persent_3)

Процент продаж топ трех товаров 0.6014215418261345


**Задание 2.1**

Серёга: Твой выход! Какой тип валидации в данном случае нужно использовать?

In [11]:
events.itemid.value_counts(normalize=True)

itemid
187946    1.237981e-03
461686    1.080512e-03
5411      8.468485e-04
370653    6.726894e-04
219512    6.530965e-04
              ...     
357730    3.628314e-07
62768     3.628314e-07
404071    3.628314e-07
145333    3.628314e-07
177353    3.628314e-07
Name: proportion, Length: 235061, dtype: float64

## 1. Постановка задачи

### 1.1. Уточнение бизнес-постановки задачи

Повысить прибыль от продаж в интернет-магазине Заказчика на 20%. С этой целью построить рекомендательную систему товаров для выдачи топ трех товаров, рекомендуемых пользователю. 

### 1.2. Выбор технических метрик качества

В соответствии с поставленой выше задачей в качестве метрики была выбрана метрика Precision@3.

### 1.3. Импорт и первичный анализ данных

**Датасет events**

In [12]:
events = pd.read_csv('data\events.csv.zip')
events.sample(5)

,timestamp,visitorid,event,itemid,transactionid
2178476,1436309891769,676380,view,364437,NaN
2631417,1437963753909,824815,view,81615,NaN
1532967,1430955835806,864733,view,91755,NaN
671876,1438723004778,129131,view,337480,NaN
1394429,1442198448476,908373,view,105815,NaN


In [13]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [15]:
events.describe(include='all')

,timestamp,visitorid,event,itemid,transactionid
count,2.756101e+06,2.756101e+06,2756101,2.756101e+06,22457.000000
unique,NaN,NaN,3,NaN,NaN
top,NaN,NaN,view,NaN,NaN
freq,NaN,NaN,2664312,NaN,NaN
mean,1.436424e+12,7.019229e+05,NaN,2.349225e+05,8826.497796
std,3.366312e+09,4.056875e+05,NaN,1.341954e+05,5098.996290
min,1.430622e+12,0.000000e+00,NaN,3.000000e+00,0.000000
25%,1.433478e+12,3.505660e+05,NaN,1.181200e+05,4411.000000
50%,1.436453e+12,7.020600e+05,NaN,2.360670e+05,8813.000000
75%,1.439225e+12,1.053437e+06,NaN,3.507150e+05,13224.000000


Проверим на пропуски.

In [16]:
events.isna().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

Проверим есть ли пропуски transactionid при условии события "transaction".

In [17]:
events[events.event=='transaction'].transactionid.isna().sum()

0

Проверим на дублирование строк.

In [22]:
events.duplicated().sum()

460

In [25]:
events['duplicate'] = events.duplicated(keep=False)
events[events['duplicate']==True].head(20)

,timestamp,visitorid,event,itemid,transactionid,duplicate
13734,1433180781440,1045411,addtocart,379647,NaN,True
22404,1433180781440,1045411,addtocart,379647,NaN,True
24476,1433274223925,366538,addtocart,252068,NaN,True
25508,1433265100661,198153,addtocart,48715,NaN,True
27095,1433270868154,1268755,view,60980,NaN,True
34465,1433274223925,366538,addtocart,252068,NaN,True
34578,1433277134682,555487,addtocart,397642,NaN,True
39738,1433265100661,198153,addtocart,48715,NaN,True
41774,1433270868154,1268755,view,60980,NaN,True
42469,1433277134682,555487,addtocart,397642,NaN,True


Удаляем дублирующиеся строки.

In [28]:
events.drop_duplicates(inplace=True)

In [29]:
events[events['duplicate']==True].head(20)

,timestamp,visitorid,event,itemid,transactionid,duplicate
13734,1433180781440,1045411,addtocart,379647,NaN,True
24476,1433274223925,366538,addtocart,252068,NaN,True
25508,1433265100661,198153,addtocart,48715,NaN,True
27095,1433270868154,1268755,view,60980,NaN,True
34578,1433277134682,555487,addtocart,397642,NaN,True
71466,1433463640728,497546,addtocart,278319,NaN,True
82954,1433475871737,1159733,addtocart,104116,NaN,True
83143,1433463912411,263168,addtocart,436219,NaN,True
94054,1433549575965,952199,addtocart,61273,NaN,True
113491,1433611737367,10162,addtocart,146661,NaN,True


In [32]:
events.drop('duplicate', axis=1, inplace=True)
events.head(3)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN


**Датасет item_properties**

In [18]:
item_properties_1 = pd.read_csv('data\item_properties_part1.csv')
item_properties_2 = pd.read_csv('data\item_properties_part2.csv')
item_properties =pd.concat([item_properties_1, item_properties_2], axis=0)
item_properties.sample(5)

,timestamp,itemid,property,value
3754249,1433646000000,313198,468,272976
5646040,1433041200000,163985,202,655083
5779626,1433646000000,26655,202,150169 820477 437265 556393 871262 183183 2363...
1429184,1433646000000,5034,283,61706 920988 639039
8440963,1435460400000,299732,categoryid,1343


In [35]:
item_properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20275902 entries, 0 to 9275902
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  int64 
 1   itemid     int64 
 2   property   object
 3   value      object
dtypes: int64(2), object(2)
memory usage: 773.5+ MB


In [36]:
item_properties.describe(include='all')

,timestamp,itemid,property,value
count,2.027590e+07,2.027590e+07,20275902,20275902
unique,NaN,NaN,1104,1966868
top,NaN,NaN,888,769062
freq,NaN,NaN,3000398,1537247
mean,1.435157e+12,2.333904e+05,NaN,NaN
std,3.327798e+09,1.348452e+05,NaN,NaN
min,1.431227e+12,0.000000e+00,NaN,NaN
25%,1.432436e+12,1.165160e+05,NaN,NaN
50%,1.433646e+12,2.334830e+05,NaN,NaN
75%,1.437880e+12,3.503040e+05,NaN,NaN


Проверяем на пропуски.

In [33]:
item_properties.isna().sum()

timestamp    0
itemid       0
property     0
value        0
dtype: int64

Проверяем на дублирование строк.

In [34]:
item_properties.duplicated().sum()

0

**Датасет category_tree**

In [37]:
category_tree = pd.read_csv('data\category_tree.csv')
category_tree.sample(5)

,categoryid,parentid
986,1302,1600.0
79,186,865.0
671,672,1120.0
1645,551,766.0
553,1,854.0


In [38]:
category_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categoryid  1669 non-null   int64  
 1   parentid    1644 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 26.2 KB


In [39]:
category_tree.describe()

,categoryid,parentid
count,1669.000000,1644.000000
mean,849.285201,847.571168
std,490.195116,505.058485
min,0.000000,8.000000
25%,427.000000,381.000000
50%,848.000000,866.000000
75%,1273.000000,1291.000000
max,1698.000000,1698.000000


Проверяем на пропуски.

In [40]:
category_tree.isna().sum()

categoryid     0
parentid      25
dtype: int64

Пока предлагаю оставить пропуск так, как это может означать, что и категории нет родительской.

Проверяем на дублирование строк.

In [41]:
category_tree.duplicated().sum()

0

## 2. Исследование данных

### 2.1. Описание структуры данных

**events — датасет с событиями.**   
**Колонки:**  
*timestamp* — время события  
*visitorid* — идентификатор пользователя  
*event* — тип события  
    Типы событий:  
    *view* (просмотр карточки товара)  
    *addtocart* (добавление в корзину)  
    *transaction* (покупка)
*itemid* — идентификатор объекта  
*transactionid* — идентификатор транзакции, если она проходила  


**category_tree — файл с деревом категорий (можно восстановить дерево).**  
**Колонки:**    
*category_id* — идентификатор категорий  
*parent_id* — идентификатор родительской категории 


**item_properties — файл с свойствами товаров.**  
**Колонки:**  
*timestamp* — момент записи значения свойства  
*item_id* — идентификатор объекта  
*property* — свойство, кажется, они все, кроме категории, захешированы  
*value* — значение свойства  

### 2.2. Статистический анализ данных

## 3. Создание факторов для модели

### 3.1. Генерация факторов, связанных с айтемами

### 3.2. Генерация факторов айтем-юзер

## 4. Проведение экспериментов

### 4.1. Коллаборативная фильтрация

### 4.2. Факторизационные машины

### 4.3. XGBoost для задачи классификации

## 5. Создание MVP

### 5.1.Проектирование API сервиса

### 5.2. Создание веб-сервиса с моделью

## 6. Контейнеризация

### 6.1. Создание Docker-контейнера с сервисом